# Visualize Parameter Sweep
In this notebook we reproduce the sweep summary figures, loading precomputed data to show the best median reconstruction distances that can be achieved over a large grid of diffusion constants, sequencing depths, and satellite densities.

In [ ]:
import anndata
import umap
import sys
import scanpy
import numpy as np
from matplotlib import pyplot as plt
import json

sys.path.append("../functions/")
import alignment
import visuals

In [ ]:
# Define the parameters for the sweep
COLONIES = [25000, 50000, 100000, 250000]
SIGMAS_UM = [30, 50, 75, 100]
MEADIAN_RPB = [10, 30, 50, 70, 90, 140, 280, 555, 1110, 2225, 4445]

First we load dictionaries of the pre-computed UMAPs and ground-truth coordinates. Since the sBC counts only give information about relative position, the embedding may be translated, shrunken, expanded, rotated, or flipped relative to the ground truth. To compare to the ground truth, we optimize for the best alignment of a simple linear transformation of the embedding. So additionally we align each UMAP to the ground truth.

In [ ]:
# Load the precomputed UMAPs and ground-truth coordinates
raw_umaps = json.load()
ground_truth = json.load()

In [ ]:
# Align each UMAP to the ground truth
umaps = {}

for rpb in MEDIAN_RPBS:
    for s in SIGMAS_UM:
        for c in COLONIES:
            umaps['{}-{}-{}'.format(rpb, s, c)] = 